<a href="https://colab.research.google.com/github/renilJoseph/MC_Assignment_Project/blob/master/MC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MC Project EEG data classifier

**Objective** : In this notebook we will be training a Neural Network model that will predict the user given their eeg data. 


Here we have the data for 106 subjects with 3 recordings each. Each recording is 2 minutes long and the sensor is capturing data at a frequency of 160Hz. So each recording is a vector of size 19200. Since our data is temporal and can be visualized as a sequence we can use a convolutional neural network. Traditionally CNNs are used for image classification tasks, where it is able to 
figure out spatial relationships between pixels.

More recently CNNs are starting to be used with time series data as well, where they are showing to be very successfull. LSTMs and RNNs are the class of neural networks that are commonly used for temporal data. When we tried our model with LSTM model it have very poor results (<10 % accuracy). CNNs aldo have the added advantage that they can be easily ported to mobiles, as Tensorflow currently does not support exporting LSTM or RNNs to mobile.

**Note**: Currently we do not have any test or validation set since we have very
few data. Once we had a few more data, it is recommended to use a validation set and test set.

**Pre-requisites**

* Scipy [for loading Matlab file]
* Numpy [for numerical computaion]
* Keras - tensorflow [for neural network]

*Authors* : Krishna Babuji, Renil Joseph, Leroy Jacob Vargis, Gopikrishnan Ravindran Nair


## Imports

Here we will import all the necessary libraries for training the model.

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import scipy.io

import numpy as np
import pandas as pd

Next  let's download the dataset if it's not already present.

In [0]:
!wget -Nq 'https://raw.githubusercontent.com/renilJoseph/MC_Assignment_Project/master/project/EEGDataset1.mat'

Once we have the data we will transform the data into a pandas dataframe for ease of use.

In [3]:
mat = scipy.io.loadmat('EEGDataset1.mat')
print(f'Mat shape : {mat["Raw_Data"].shape}')

eeg_df1 = pd.DataFrame(mat['Raw_Data'][:, 0, :])
eeg_df1['id'] = eeg_df1.index
eeg_df2 = pd.DataFrame(mat['Raw_Data'][:, 1, :])
eeg_df2['id'] = eeg_df2.index
eeg_df3 = pd.DataFrame(mat['Raw_Data'][:, 2, :])
eeg_df3['id'] = eeg_df3.index

eeg_df = pd.concat([eeg_df1, eeg_df2, eeg_df3], ignore_index=True)
print(f'eeg_df shape : {eeg_df.shape}')

Mat shape : (106, 3, 19200)
eeg_df shape : (318, 19201)


Here we can see that each vector is 19201 long(with the additional Id column).
A reasonable assumption to make is that our predictions won't change much if we change the sampling rate from 160Hz to 80 Hz. Reducing the sampling rate will mean that our LSTM need not handle such a long vector. 

Another option that we have yet to try is just use the first half of the vector, that is use the full sixty seconds with 160 Hz instead of the 120 seconds with 80Hz. But the key idea here is to reduce the dimensions without loosing too much data.

In [0]:
#Down sample from 120Hz to 60Hz
col_drops = [ x for x in eeg_df.columns if x != 'id' and int(x)%2 ==1 ]
eeg_df.drop(col_drops, axis=1, inplace=True)

In [5]:
X_train, y_train = eeg_df.drop(['id'], axis=1), eeg_df['id']
X_train = np.expand_dims(X_train, axis=2)
#X_train = normalize(X_train, axis=0, order=2)
y_train = to_categorical(y_train)
print(X_train.shape, y_train.shape)

(318, 9600, 1) (318, 106)


#Model Definition & Training

In [6]:
model = Sequential()
model.add(LSTM(64, input_shape=(9600,1)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(106, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Train...')

print(model.summary())
model.fit(X_train, y_train, batch_size=64, shuffle=True, epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                16896     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 106)               27242     
Total params: 60,778
Trainable params: 60,778
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 318 samples
Epoch 1/10
318/318 [====================

In [0]:
model.save('lstm_eeg.h5')

Accuracy of the model is not impressive, but we can see that it is better than  a naive model which is random guessing which would be at ~0.01.

If we had more EEG data for these same users we could have created a validation set and tried to tune the hyperparameters more. Since we have only 3 recordings per user, it does not make much sense to create a train and validation set. 

Let's save this model for our predictions.

In [8]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(9600,1)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(106, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Train...')

print(model.summary())
model.fit(X_train, y_train, batch_size=64, shuffle=True, epochs=7)

Train...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 9598, 64)          256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 9596, 32)          6176      
_________________________________________________________________
dropout_1 (Dropout)          (None, 9596, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4798, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 153536)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               39305472  
_________________________________________________________________
dense_3 (Dense)              (None, 106)     

In [0]:
model.save('cnn_eeg.h5')

In [10]:
!toco \
  --output_file=cnn_eeg.tflite\
  --keras_model_file=cnn_eeg.h5

W1207 02:03:50.061067 139719327594368 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1207 02:03:50.061634 139719327594368 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1207 02:03:50.213816 139719327594368 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with